In [2]:
import math
import os

import cv2
from PIL import Image
import random
import numpy as np
from utils.detect_line import detect_lines
from utils.hot import hot_detect_line
import pandas as pd

import json

In [3]:
ori_img = cv2.imread('../static/img/4.png')
lines = hot_detect_line('../static/img/4.png', None, None, 0, path='../static/record/')

正在重载4.png[None:None, None:None]


In [4]:
# 颜色过滤
def color_filter(lines, color):
    return [line for line in lines if tuple(line[2]) != color]
def _line_length(line):
    return math.sqrt((line[0][0] - line[1][0]) ** 2 + (line[0][1] - line[1][1]) ** 2)
#是竖线
def _is_vertical(line):
    return line[0][0] - line[1][0] == 0
def _is_horizontal(line):
    return line[0][1] - line[1][1] == 0
# 斜线和直线分层
def line_filter(lines):
    lines_a = []
    lines_b = []
    for line in lines:
        if _is_vertical(line) or _is_horizontal(line):
            lines_a.append(line)
        else:
            lines_b.append(line)
    return lines_a, lines_b
# 去除黑色
lines_a = color_filter(lines, (0, 0, 0))
# 分离水平垂直
lines_a, lines_b = line_filter(lines_a)

lines = pd.DataFrame([[line[0][1], line[0][0], line[1][1], line[1][0], tuple(line[2]), int(_line_length(line))] for line in lines_a], columns=['x1', 'y1', 'x2', 'y2', 'color', 'length'])
lines

,x1,y1,x2,y2,color,length
0,1467,2154,1481,2154,"(0, 255, 0)",14
1,1467,2154,1481,2154,"(0, 255, 0)",14
2,2028,2154,2042,2154,"(0, 255, 0)",14
3,1458,2155,1466,2155,"(0, 255, 0)",8
4,1482,2155,1490,2155,"(0, 255, 0)",8
...,...,...,...,...,...,...
26486,18489,24809,18489,24817,"(0, 255, 0)",8
26487,18675,24785,18675,24793,"(0, 255, 0)",8
26488,18675,24809,18675,24817,"(0, 255, 0)",8
26489,18676,24794,18676,24808,"(0, 255, 0)",14


In [26]:
le = 469
lines_len = lines[lines.length == le]
lines_len = lines_len[lines_len.color == (128, 128, 128)]
# 只保留坐标
lines_len = lines_len[['x1', 'y1', 'x2', 'y2']]

In [27]:
lines_len

,x1,y1,x2,y2
134,1737,3625,1737,4094
136,1755,3625,1755,4094
1019,1737,10666,1737,11135
1024,1755,10666,1755,11135
1194,1737,13355,1737,13824
...,...,...,...,...
23679,16768,19360,16768,19829
23680,16787,19360,16787,19829
24388,16750,20867,16750,21336
24389,16768,20867,16768,21336


In [28]:
lines_len.x1.value_counts()

x1
4190     12
2934     12
2953     12
3562     12
4818     12
         ..
6093      4
1774      4
16750     4
16787     4
16768     3
Name: count, Length: 69, dtype: int64

In [29]:
lines_len.y1.value_counts()

y1
20867    69
19360    63
13355    62
11526    57
17531    50
22696    46
14862    45
16691    40
10666    20
3625     19
6741     10
7550      9
Name: count, dtype: int64

In [30]:
lines_len.x2.value_counts()

x2
4190     12
2934     12
2953     12
3562     12
4818     12
         ..
6093      4
1774      4
16750     4
16787     4
16768     3
Name: count, Length: 69, dtype: int64

In [31]:
lines_len.y2.value_counts()

y2
21336    69
19829    63
13824    62
11995    57
18000    50
23165    46
15331    45
17160    40
11135    20
4094     19
7210     10
8019      9
Name: count, dtype: int64

In [34]:
x1 = lines_len.x1.value_counts() / len(lines_len)
x1

x1
4190     0.024490
2934     0.024490
2953     0.024490
3562     0.024490
4818     0.024490
           ...   
6093     0.008163
1774     0.008163
16750    0.008163
16787    0.008163
16768    0.006122
Name: count, Length: 69, dtype: float64

In [35]:
y1 = lines_len.y1.value_counts() / len(lines_len)
y1

y1
20867    0.140816
19360    0.128571
13355    0.126531
11526    0.116327
17531    0.102041
22696    0.093878
14862    0.091837
16691    0.081633
10666    0.040816
3625     0.038776
6741     0.020408
7550     0.018367
Name: count, dtype: float64